<a href="https://colab.research.google.com/github/SARA3SAEED/aws-ML/blob/main/03_Assigning_Roles_Role_Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 3: Assigning Roles (Role Prompting)

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
# Import python's built-in regular expression library
import re
import boto3
from botocore.exceptions import ClientError
import json

# Import the hints module from the utils package
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [ ]:
def get_completion(prompt, system_prompt=None):
    inference_config = {
        "temperature": 0.0,
        "maxTokens": 500
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

---

## Lesson

Continuing on the theme of Claude having no context aside from what you say, it's sometimes important to **prompt Claude to inhabit a specific role (including all necessary context)**. This is also known as role prompting. The more detail to the role context, the better.

**Priming Claude with a role can improve Claude's performance** in a variety of fields, from writing to coding to summarizing. It's like how humans can sometimes be helped when told to "think like a ______". Role prompting can also change the style, tone, and manner of Claude's response.

**Note:** Role prompting can happen either in the system prompt or as part of the User message turn.

### Examples

In the example below, we see that without role prompting, Claude provides a **straightforward and non-stylized answer** when asked to give a single sentence perspective on skateboarding.

However, when we prime Claude to inhabit the role of a cat, Claude's perspective changes, and thus **Claude's response tone, style, content adapts to the new role**.

**Note:** A bonus technique you can use is to **provide Claude context on its intended audience**. Below, we could have tweaked the prompt to also tell Claude whom it should be speaking to. "You are a cat" produces quite a different response than "you are a cat talking to a crowd of skateboarders.

Here is the prompt without role prompting in the system prompt:

In [ ]:
# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT))

Skateboarding is an exhilarating and creative form of self-expression that combines athleticism, artistry, and a unique sense of freedom and community.


Here is the same user question, except with role prompting.

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a cat."

# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

*meows* Skateboarding looks like a fun way for humans to zoom around on four wheels while balancing precariously, though I much prefer chasing after feathers and napping in sunbeams.


You can use role prompting as a way to get Claude to emulate certain styles in writing, speak in a certain voice, or guide the complexity of its answers. **Role prompting can also make Claude better at performing math or logic tasks.**

For example, in the example below, there is a definitive correct answer, which is yes. However, Claude gets it wrong and thinks it lacks information, which it doesn't:

In [ ]:
# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT))

To solve this problem, we need to analyze the given information and determine if a married person is looking at an unmarried person.

Given information:
- Jack is looking at Anne.
- Anne is looking at George.
- Jack is married.
- George is not married.
- We don't know if Anne is married or not.

To determine if a married person is looking at an unmarried person, we need to identify the relationship between the people involved.

1. Jack is a married person, and he is looking at Anne.
2. We don't know if Anne is married or not, so we cannot conclude whether a married person is looking at an unmarried person in this case.

However, we can say that a married person (Jack) is looking at a person (Anne) whose marital status is unknown.

Therefore, the answer is: We cannot conclude whether a married person is looking at an unmarried person based on the given information, as we don't know Anne's marital status.


Now, what if we **prime Claude to act as a logic bot**? How will that change Claude's answer?

It turns out that with this new role assignment, Claude gets it right. (Although notably not for all the right reasons)

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."

# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

To solve this problem, we need to analyze the given information and determine the relationship between the people.

Given information:
- Jack is looking at Anne.
- Anne is looking at George.
- Jack is married.
- George is not married.
- We don't know if Anne is married.

To determine if a married person is looking at an unmarried person, we need to identify the marital status of the people involved in the scenario.

We know that:
- Jack is married.
- George is not married.

Since Anne's marital status is unknown, we cannot definitively conclude whether a married person is looking at an unmarried person.

However, we can make the following observations:
- Jack, who is married, is looking at Anne, whose marital status is unknown.
- Anne, whose marital status is unknown, is looking at George, who is not married.

Therefore, based on the given information, it is possible that a married person (Jack) is looking at an unmarried person (George), but we cannot say this with certainty due to th

**Note:** What you'll learn throughout this course is that there are **many prompt engineering techniques you can use to derive similar results**. Which techniques you use is up to you and your preference! We encourage you to **experiment to find your own prompt engineering style**.

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 3.1 - Math Correction](#exercise-31---math-correction)

### Exercise 3.1 - Math Correction
In some instances, **Claude may struggle with mathematics**, even simple mathematics. Below, Claude incorrectly assesses the math problem as correctly solved, even though there's an obvious arithmetic mistake in the second step. Note that Claude actually catches the mistake when going through step-by-step, but doesn't jump to the conclusion that the overall solution is wrong.

Modify the `PROMPT` and / or the `SYSTEM_PROMPT` to make Claude grade the solution as `incorrectly` solved, rather than correctly solved.


In [ ]:
# System prompt - if you don't want to use a system prompt, you can leave this variable set to an empty string
SYSTEM_PROMPT = ""

# Prompt
PROMPT = """Is this equation solved correctly below?

2x - 3 = 9
2x = 6
x = 3"""

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    if "incorrect" in text or "not correct" in text.lower():
        return True
    else:
        return False

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

Yes, the equation is solved correctly.

Let's go through the steps:

1. 2x - 3 = 9
2. Add 3 to both sides to isolate the variable term:
   2x - 3 + 3 = 9 + 3
   2x = 12
3. Divide both sides by 2 to solve for x:
   2x/2 = 12/2
   x = 6

The solution is correct. The final value of x is 6.

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: False


❓ If you want a hint, run the cell below!

In [ ]:
print(hints.exercise_3_1_hint)

The grading function in this exercise is looking for an answer that includes the words "incorrect" or "not correct".
Give Claude a role that might make Claude better at solving math problems!


### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT))

Skateboarding is an exhilarating and creative form of self-expression that combines athleticism, artistry, and a unique sense of freedom and community.


In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a cat."

# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

*meows* Skateboarding looks like a fun way for humans to zoom around on four wheels while balancing precariously, though I much prefer chasing feathers and napping in sunbeams.


In [ ]:
# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT))

To solve this problem, we need to analyze the given information and determine if a married person is looking at an unmarried person.

Given information:
- Jack is looking at Anne.
- Anne is looking at George.
- Jack is married.
- George is not married.
- We don't know if Anne is married or not.

To determine if a married person is looking at an unmarried person, we need to identify the relationship between the people involved.

1. Jack is married and is looking at Anne.
2. We don't know if Anne is married or not.
3. Anne is looking at George, who is not married.

Therefore, a married person (Jack) is looking at an unmarried person (George), even though we don't know the marital status of the person (Anne) in the middle.

So, the answer is yes, a married person (Jack) is looking at an unmarried person (George).


In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."

# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

To solve this problem, we need to analyze the given information and determine the relationship between the people.

Given information:
- Jack is looking at Anne.
- Anne is looking at George.
- Jack is married.
- George is not married.
- We don't know if Anne is married.

To determine if a married person is looking at an unmarried person, we need to identify the marital status of the people involved in the scenario.

We know that:
- Jack is married.
- George is not married.

However, we don't know the marital status of Anne. Since we don't have enough information to determine if Anne is married or not, we cannot conclusively say whether a married person is looking at an unmarried person.

Therefore, the answer is: We cannot determine if a married person is looking at an unmarried person based on the given information.
